In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [2]:
import os
from langchain.chains import RetrievalQA
from langchain.chat_models import BedrockChat
from langchain.embeddings import BedrockEmbeddings
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

IS_DOCS_ALREADY_EMBEDDED = False

RAW_DATA_DIR = "../raw_data/company/20240305"
FILENAME = "1709599445799.pdf"

VECTOR_STORE_DIR = "../vector_store"
INDEX_NAME = "manual"

file_path = os.path.join(RAW_DATA_DIR, FILENAME)

In [3]:
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)
loader = PyPDFLoader(file_path)

raw_docs = loader.load()
proc_docs = text_splitter.split_documents(raw_docs)

In [4]:
avg_doc_length = lambda docs: sum([len(doc.page_content) for doc in docs]) // len(docs)

avg_char_count_raw = avg_doc_length(raw_docs)
avg_char_count_proc = avg_doc_length(proc_docs)

print(
    f"Average length among {len(raw_docs)} documents loaded is {avg_char_count_raw} characters."
)
print(
    f"After the split we have {len(proc_docs)} documents more than the original {len(raw_docs)}."
)
print(
    f"Average length among {len(proc_docs)} documents (after split) is {avg_char_count_proc} characters."
)

Average length among 6 documents loaded is 1718 characters.
After the split we have 15 documents more than the original 6.
Average length among 15 documents (after split) is 731 characters.


In [5]:
%%time
embeddings = BedrockEmbeddings(
    region_name="us-east-1", model_id="amazon.titan-embed-text-v1"
)

if IS_DOCS_ALREADY_EMBEDDED:
    vector_store = FAISS.load_local(
        os.path.join(VECTOR_STORE_DIR, INDEX_NAME), embeddings
    )

else:
    vector_store = FAISS.from_documents(proc_docs, embeddings)
    vector_store.save_local(os.path.join(VECTOR_STORE_DIR, INDEX_NAME))

retriever = vector_store.as_retriever()

CPU times: user 232 ms, sys: 52.6 ms, total: 285 ms
Wall time: 10.3 s


In [6]:
llm = BedrockChat(
    region_name="us-east-1",
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.0, "max_tokens": 1024},
)

In [7]:
system_template = """Use the following pieces of context to answer the user's question. If you don't know the answer,
just say that you don't know, don't try to make up an answer. Answer in Korean.
----------------
{context}"""

human_template = "{question}"

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template),
]

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": ChatPromptTemplate.from_messages(messages)},
)

In [8]:
result = qa_chain({"query": "삼성생명 목표 주가는 얼마야? 그 이유는?"})
print(result["result"].strip())

/Users/youngmki/anaconda3/envs/wealth-whisper/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


삼성생명의 목표주가는 125,000원으로 기존 대비 24.5% 상향 조정되었습니다. 목표주가 상향 조정 이유는 2024년부터 BPS(Book Price per Share, 주당순자산가치)를 적용하고 할인율을 조정했기 때문입니다. 이에 따라 삼성생명의 목표 PBR(주가순자산비율)은 0.45배가 적용되었습니다.


In [9]:
result = qa_chain({"query": "삼성생명 최근 주가 수익률 알려줘"})
print(result["result"].strip())

삼성생명의 최근 주가수익률은 다음과 같습니다.

1개월 수익률: 47.5%
3개월 수익률: 39.6% 
6개월 수익률: 43.8%
12개월 수익률: 42.3%

상대수익률(KOSPI 대비)
1개월: 39.6%
3개월: 33.1%  
6개월: 38.9%
12개월: 29.9%

최근 1년간 삼성생명 주가는 KOSPI 지수를 상회하는 높은 수익률을 기록했습니다. 특히 단기(1개월, 3개월, 6개월)에서 40% 내외의 높은 절대수익률을 보였습니다.


In [10]:
result = qa_chain({"query": "삼성생명 상품별 CSM 배수 최근 추세 알려줘"})
print(result["result"].strip())

삼성생명의 상품별 CSM 배수 최근 추세에 대해서는 정확한 정보를 갖고 있지 않습니다. 하지만 제공된 자료에서 일부 내용을 확인할 수 있습니다.

그림 3에서 2022년 4분기 기준 상품별 CSM 배수를 보여주고 있습니다. 
- 사망보장성 상품의 CSM 배수는 약 0.8배 수준
- 건강보장성 상품의 CSM 배수는 약 1.2배 수준 
- 연금/저축성 상품의 CSM 배수는 약 0.4배 수준

이를 통해 건강보장성 상품의 CSM 배수가 가장 높고, 연금/저축성 상품이 가장 낮은 것으로 나타납니다. 하지만 시점별 추이는 알려지지 않아 최근 추세를 파악하기는 어렵습니다. 상품별 CSM 배수의 변동 요인과 향후 전망 등에 대해서는 추가 정보가 필요할 것 같습니다.
